# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [4]:
from xml.etree import ElementTree as ET
import pandas as pd
%pylab inline

Populating the interactive namespace from numpy and matplotlib


## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [5]:
doc = ET.parse( 'mondial_database.xml' )
document_tree1 = ET.parse('mondial_database.xml')

In [ ]:
# print names of all countries
for child in document_tree.getroot():
    print child.find('name').text

In [ ]:
for child in document_tree1.getroot():
    print child.find('name').text

In [ ]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print '* ' + element.find('name').text + ':',
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print capitals_string[:-2]

In [ ]:
for element in document_tree1.iterfind('country'):
    print '* ' + element.find('name').text + ':',
    capitals_string = '' 
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print capitals_string[:-2]

****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

## Number 1

In [28]:
doc = ET.parse( 'mondial_database.xml' )
# create empty DF with desired column names
df_IM = pd.DataFrame(columns=['country', 'infant_mortality'])

#iterate through each country
for country in doc.getiterator('country'):
    #within each country, iterate through the various children of the country branch
    for element in list(country):
        if element.tag == 'infant_mortality':
            #append the country and paired IM to our dataframe
            df_IM = df_IM.append({
                'country': country.find('name').text,
                'infant_mortality': element.text,
            }, ignore_index=True)

#make numeric so we can sort
df_IM.infant_mortality = pd.to_numeric(df_IM.infant_mortality)           
#show top 10
df_IM.sort_values('infant_mortality').head(10)


,country,infant_mortality
36,Monaco,1.81
90,Japan,2.13
109,Bermuda,2.48
34,Norway,2.48
98,Singapore,2.53
35,Sweden,2.60
8,Czech Republic,2.63
72,Hong Kong,2.73
73,Macao,3.13
39,Iceland,3.15


## Number 2

In [36]:
#bring in data
doc = ET.parse( 'mondial_database.xml' )
#make a dataframe for population
df_pop = pd.DataFrame(columns=['country', 'population', 'year'])

#repeat same process as above, but this time we pull population and year of observation
for country in doc.getiterator('country'):
    for element in list(country):
        if element.tag == 'population':
            df_pop = df_pop.append({
                'country': country.find('name').text,
                'population': element.text,
                'year': element.attrib['year']
            }, ignore_index=True)

#make numeric so we can sort
df_pop.population = pd.to_numeric(df_pop.population)           
            
#sort on population, dropping older years
df_pop.sort_values(['population','year'], ascending = False).drop_duplicates('country', keep = 'first').head(10)


,country,population,year
578,China,1360720000,2013
688,India,1210854977,2011
1155,United States,318857056,2014
854,Indonesia,252124458,2014
1652,Brazil,202768562,2014
595,Pakistan,173149306,2010
1858,Nigeria,164294516,2011
665,Bangladesh,149772364,2011
225,Russia,148178487,1997
930,Japan,128057352,2010


## Number 3

In [37]:
doc = ET.parse( 'mondial_database.xml' )
# dataframe for fraction of each ethnic group in country
df_group = pd.DataFrame(columns=['country', 'group', 'percent_of_pop'])

#iterate through countries, and then ethnic groups, pulling info on country, group and percent of population
for country in doc.getiterator('country'):
    for element in list(country):
        if element.tag == 'ethnicgroup':
            df_group = df_group.append({
                'country': country.find('name').text,
                'group': element.text,
                'percent_of_pop': element.attrib['percentage']
            }, ignore_index=True)
 
            

# convert percentage to numeric
df_group['percent_of_pop'] = pd.to_numeric(df_group['percent_of_pop'])

df_pop = pd.DataFrame(columns=['country', 'population', 'year'])

# repeat same process as above, but this time we pull population and year of observation
for country in doc.getiterator('country'):
    for element in list(country):
        if element.tag == 'population':
            df_pop = df_pop.append({
                'country': country.find('name').text,
                'population': element.text,
                'year': element.attrib['year']
            }, ignore_index=True)
            
#convert population to numeric so we can perform operation on it
df_pop['population'] = pd.to_numeric(df_pop['population'])
#sort and merge population and ethnic group on country
df_final = df_pop.sort_values('population').merge(df_group, on = 'country')

#multiply population by the percentage to get size of ethnic group
df_final['size_of_group'] = df_final['percent_of_pop']/100 * df_final['population']
            
#sort on size of group to get largest 10, drop, all duplicate countries, but keep the first (newest) data
df_final.sort_values('size_of_group', ascending = False).drop_duplicates('country',keep='first').head(10)

,country,population,year,group,percent_of_pop,size_of_group
5791,China,1360720000,2013,Han Chinese,91.50,1.245059e+09
5782,India,1210854977,2011,Indo-Aryan,72.00,8.718156e+08
5744,United States,318857056,2014,European,79.96,2.549581e+08
5520,Nigeria,164294516,2011,African,99.00,1.626516e+08
5431,Bangladesh,149772364,2011,Bengali,98.00,1.467769e+08
5661,Japan,128057352,2010,Japanese,99.40,1.272890e+08
5707,Russia,148178487,1997,Russian,79.80,1.182464e+08
5648,Indonesia,252124458,2014,Javanese,45.00,1.134560e+08
5605,Brazil,202768562,2014,European,53.70,1.088867e+08
2411,Philippines,92337852,2007,Malay,95.50,8.818265e+07


## Number 4

In [6]:
doc = ET.parse( 'mondial_database.xml' )

# dataframe for relevant info
df_elev = pd.DataFrame(columns=["river", "length", "countries"])

#pull all the rivers and length of those river, wanted to pull country code as well, but unsuccesful
for river in doc.getiterator("river"):
    for element in list(river):
        if element.tag == "length":
            df_elev = df_elev.append({
                "river": river.find("name").text,
                "length": element.text
            }, ignore_index=True)
#trying to pull river, and country code here, to no avail of course
        if element.tag == "country":
            df_elev = df_elev.append({
                "river": river.find("name").text,
                "countries": element.attrib['river']
            }, ignore_index=True)

#Need to convert to numeric so we can sort
df_elev.length = pd.to_numeric(df_elev.length)       
#show the longest river
df_elev.sort('length', ascending = False).head(1)
            
            
           

##considered bringing in another df with country data, using river within country and then maybe do a merge
##still not working
#riv_con = pd.DataFrame(columns=["country",'river'])
#for country in doc.getiterator('country'):
    #for element in list(country):
        #if element.tag == 'located_at':
            #riv_con = riv_con.append({
                #'country': country.text,
                #'river': element.attrib['river']
                #'river': element.attrib['river']
            #}, ignore_index=True)
#riv_con.head()              
         
            







/Users/jackdwyer/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:24: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,river,length,countries
174,Amazonas,6448.0,NaN


In [33]:
doc = ET.parse( 'mondial_database.xml' )

# dataframe for relevant info
df_lake = pd.DataFrame(columns=["lake", "area", "countries"])

#pull all the lakes and areas of those lakes, wanted to pull country code as well, but unsuccesful
for lake in doc.getiterator("lake"):
    for element in list(lake):
         if element.tag == 'country':
            df_lake = df_lake.append({
                "countries": element.text
            },ignore_index = True)
        if element.tag == "area":
            df_lake = df_lake.append({
                "lake": lake.find("name").text,
                "area": element.text
            }, ignore_index=True)


#convert area to numeric so we can sort easily
df_lake['area'] = pd.to_numeric(df_lake['area'])

#show most expansive lake
df_lake.sort_values('area', ascending=False).head(1)


#If the caspian is a lake, then Everest is a hill



,lake,area,countries
54,Caspian Sea,386400.0,NaN


In [8]:
doc = ET.parse( 'mondial_database.xml' )

# dataframe made to hold all the relevant info we're pulling
df_air = pd.DataFrame(columns=["airport", "elevation", "countries"])

#pull all the airports and elevations of those airports, wanted to pull country code as well, but unsuccesful
for airport in doc.getiterator("airport"):
    for element in list(airport):
        if element.tag == 'country':
            df_air = df_air.append({
                "countries": element.text
            },ignore_index = True)
        if element.tag == "elevation":
            df_air = df_air.append({
                "airport": airport.find("name").text,
                "elevation": element.text
            }, ignore_index=True)

#convert elevation to numeric so we can sort
df_air['elevation'] = pd.to_numeric(df_air['elevation'])
#show highest airport
df_air.sort_values('elevation', ascending = False).head(1)


,airport,elevation,countries
80,El Alto Intl,4063.0,NaN
